From Justin Johnson:

> In TensorFlow, packages like Keras, TensorFlow-Slim, and TFLearn provide higher-level abstractions over raw computational graphs that are useful for building neural networks.

> In PyTorch, the nn package serves this same purpose. The nn package defines a set of Modules, which are roughly equivalent to neural network layers. A Module receives input Tensors and computes output Tensors, but may also hold internal state such as Tensors containing learnable parameters. The nn package also defines a set of useful loss functions that are commonly used when training neural networks.

In [1]:
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Hyperparameters

In [3]:
# Training
num_epochs = 5
batch_size = 64
learning_rate = 0.001
num_classes = 100

# Dataset

In [10]:
training_data = torchvision.datasets.MNIST(root="data",
                                           download=True,
                                           transform=transforms.ToTensor(),
                                           train=True)

training_dataloader = torch.utils.data.DataLoader(training_data, 
                                                  batch_size=batch_size,
                                                  shuffle=True)

# Testing Set
testing_data = torchvision.datasets.MNIST(root="data",
                                          transform=transforms.ToTensor(),
                                          train=False)

testing_dataloader = torch.utils.data.DataLoader(testing_data,
                                                batch_size=batch_size,
                                                shuffle=False)

In [11]:
len(testing_data)

10000

# Model

In [13]:
class ConvNet(nn.Module):
    
    def __init__(self, num_classes, kernel_size=5, stride=1, padding=2):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size, stride, padding),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size, stride, padding),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        """ 
        Defines the computation performed at every call.
        
        Should be overridden by all subclasses.

        """
        output = self.layer1(x)
        output = self.layer2(output)
        output = output.reshape(output.size(0), -1)
        output = self.fc(output)
        return output

In [18]:
model = ConvNet(num_classes).to(device)

# Cost Function and Optimizer

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training

In [26]:
total_steps = len(training_dataloader)

In [30]:
for epoch in range(num_epochs):
    
    for index, batch in enumerate(training_dataloader):
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        
        predictions = model(images)
        
        # Optimize
        loss = criterion(predictions, labels)
        
        # Set the gradients of the Tensors
        # that are to be updated to zero
        optimizer.zero_grad()
        
        # Compute gradients of the loss with respect to the
        # model parameters
        loss.backward()
        
        # The step function then updates the models parameters
        optimizer.step()
        
        if (index+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, index+1, total_steps, loss.item()))
        

Epoch [1/5], Step [100/938], Loss: 0.2180
Epoch [1/5], Step [200/938], Loss: 0.0637
Epoch [1/5], Step [300/938], Loss: 0.0354
Epoch [1/5], Step [400/938], Loss: 0.2405
Epoch [1/5], Step [500/938], Loss: 0.0241
Epoch [1/5], Step [600/938], Loss: 0.1091
Epoch [1/5], Step [700/938], Loss: 0.0565
Epoch [1/5], Step [800/938], Loss: 0.0605
Epoch [1/5], Step [900/938], Loss: 0.0042
Epoch [2/5], Step [100/938], Loss: 0.0114
Epoch [2/5], Step [200/938], Loss: 0.0130
Epoch [2/5], Step [300/938], Loss: 0.0337
Epoch [2/5], Step [400/938], Loss: 0.1116
Epoch [2/5], Step [500/938], Loss: 0.1043
Epoch [2/5], Step [600/938], Loss: 0.0981
Epoch [2/5], Step [700/938], Loss: 0.0239
Epoch [2/5], Step [800/938], Loss: 0.0283
Epoch [2/5], Step [900/938], Loss: 0.0568
Epoch [3/5], Step [100/938], Loss: 0.0126
Epoch [3/5], Step [200/938], Loss: 0.0278
Epoch [3/5], Step [300/938], Loss: 0.0140
Epoch [3/5], Step [400/938], Loss: 0.0065
Epoch [3/5], Step [500/938], Loss: 0.0284
Epoch [3/5], Step [600/938], Loss:

# Testing

In [57]:
correct, total = 0, 0

for batch in testing_dataloader:
    images, labels = batch
    
    images = images.to(device)
    labels = labels.to(device)
    
    output = model(images)
    
    # Why use data?
    _, predictions = torch.max(output.data, 1)
    total += labels.size(0)
    # Use torch.Tensor.item() to get a Python number from a tensor containing a single value:
    correct += (predictions == labels).sum().item()

print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 99.12 %
